In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [2]:
import functions.parsers as pars
from functions.interpolation import interpolative_merge
from functions.cleaners import fill_timegaps
import visualization.plot as vpt

import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

## Load Data

### filepaths

In [3]:
gas_file = r"../../datastreams/FlowData/Export (2021-11-29 132252 to 2021-12-03 142723).csv" #r"data/FlowData/Export (2021-11-29 132252 to 2021-12-03 142723).csv"
xrd_file = r"../../raw/Na-Y_CBV100/combined.txt"
topas_file = r"../../refinements/Na-Y_CBV100/Resultsold.txt"
ms_file = r"../../datastreams//MS/0212202101.asc"
heater_file = r"../../datastreams/Heater/MAX002.xlsm"

### dataframes

In [4]:
df_ms = pars.ms(ms_file)
df_gas = pars.gas_system(gas_file)
df_heater = pars.heater(heater_file)
df_xrd = pars.danmax_xrd(xrd_file)
df_topas = pars.topas(topas_file)

## Modifying data

In [5]:
df_total = pars.join_xrd_topas(df_topas, df_xrd)

## Fixing data gaps

In [9]:
df_total = fill_timegaps(df_total, threshold=15)

## Interpolation

In [10]:
interpolative_merge(df1=df_total, df2=df_gas, df1_time="XRDTimeStamp", df2_time="Timestamp", columns=["FI111_PV", "FI111_SP", "FIC111_SP", "FIC111_PV", "GVV111_HS", "GVV121_HS", "GBV122_HS"], inplace=True)
interpolative_merge(df1=df_total, df2=df_heater, df1_time="XRDTimeStamp", df2_time="HistoricalTimeString", columns=["TIC100_PV", "TIC100_SP"], inplace=True)

MS_columns = [col for col in df_ms.columns[1:].to_list() if col not in ["RelTime[s]", "MSTimeStamp"]]
interpolative_merge(df1=df_total, df2=df_ms, df1_time="XRDTimeStamp", df2_time="MSTimeStamp", columns=MS_columns, floor=["Cycle"] , inplace=True)

df_total["Rel.Time/min"] = (df_total["XRDTimeStamp"] - df_total["XRDTimeStamp"][0]).dt.total_seconds()/60
df_total = df_total.set_index("Rel.Time/min").reset_index()

## Visualization

In [11]:
vpt.plot(df_total)

## To Csv

In [100]:
import os
import re

xrd_name = re.findall(r"(?<=\/)[^.]+", xrd_file)[0]
result_name = os.path.join(r"../../analysis/DanMAX_202112/", "_combined.csv")
df_total.to_csv(result_name, sep=";", decimal=",", index=False)